In [1]:
import pandas as pd
import psycopg2
from psycopg2 import sql

In [3]:
df = pd.read_csv("C:\\Users\\mdw2685\\Desktop\\Projects\\early_tests\\GBIF_downloads\\0000148-250227182400228.csv", delimiter='\t')

In [14]:
with pd.option_context('display.max_columns', 2000):
    print(df.head())

       gbifID                            datasetKey  \
0  3858213309  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
1  4133669450  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
2  3747015355  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
3  2823201352  50c9509d-22c7-4a22-a47d-8c48425ef4a7   
4  3384899122  50c9509d-22c7-4a22-a47d-8c48425ef4a7   

                                        occurrenceID   kingdom    phylum  \
0    http://www.inaturalist.org/observations/2997096  Animalia  Chordata   
1  https://www.inaturalist.org/observations/15801...  Animalia  Chordata   
2  https://www.inaturalist.org/observations/39728783  Animalia  Chordata   
3  https://www.inaturalist.org/observations/53881806  Animalia  Chordata   
4  https://www.inaturalist.org/observations/95930379  Animalia  Chordata   

        class            order     family         genus  \
0  Ascidiacea  Stolidobranchia  Styelidae     Stolonica   
1  Ascidiacea  Stolidobranchia   Pyuridae         Pyura   
2  Ascidiacea  Stolidobranchia   Py

In [60]:
# GBIF taxon .tsv filepath
taxon_fp = './backbone/Taxon.tsv'

In [6]:
# read GBIF taxon file as TSV
df = pd.read_csv(taxon_fp, delimiter='\t', on_bad_lines='skip')

NameError: name 'taxon_fp' is not defined

In [66]:
# filter by non-Chordates in Animalia
inverts_df = df.loc[(df['kingdom'] == 'Animalia') & (df['phylum'] != 'Chordata')]

In [67]:
print(inverts_df)

          taxonID                             datasetID  parentNameUsageID  \
118       8987139  c8227bb4-4143-443f-8cb2-51f9576aff14                1.0   
944             1  d7dddbf4-2cf0-4f39-9b2a-bb099caae36c                NaN   
24012     7961678  80b4b440-eaca-4860-aadf-d0dfdd3e856e                1.0   
24013     8137953  80b4b440-eaca-4860-aadf-d0dfdd3e856e                1.0   
24014     7758685  80b4b440-eaca-4860-aadf-d0dfdd3e856e                1.0   
...           ...                                   ...                ...   
7694308  11346263  dbaa27eb-29e7-4cbb-8eab-3f689cfce116          1593713.0   
7694310   1517849  7ddf754f-d193-4cc9-b351-99906754a03b          1517775.0   
7694316   5970807  7ddf754f-d193-4cc9-b351-99906754a03b          2226678.0   
7694317   2226679  7ddf754f-d193-4cc9-b351-99906754a03b          2226678.0   
7694318   5970805  7ddf754f-d193-4cc9-b351-99906754a03b          2226678.0   

         acceptedNameUsageID  originalNameUsageID  \
118       

In [68]:
# save to csv
inverts_df.to_csv('./Inverts_Taxon.csv', index=False)

<h1>NEW TABLE CREATION</h1>

In [ ]:
# create psycopg2 connection
conn = psycopg2.connect(dbname='tpwd-tests',
                       user='postgres', host='localhost', port='5432',
                       password='postgres')
cur = conn.cursor()

# begin table creation query
query = 'DROP TABLE IF EXISTS ' + 'gbif_inverts_backbone' + ';\n'
query += 'CREATE TABLE ' + 'gbif_inverts_backbone' + '('

# create table columns usings columns from GBIF csv
# TODO:
# the CHECKs are failing, as there are some values outside of those currently listed
query += 'taxonID' + ' TEXT PRIMARY KEY,\n'
query += 'datasetID' + ' TEXT,\n'
query += 'parentNameUsageID' + ' TEXT,\n'
query += 'acceptedNameUsageID' + ' TEXT,\n'
query += 'originalNameUsageID' + ' TEXT,\n'
query += 'scientificName' + ' TEXT NOT NULL,\n'
query += 'scientificNameAuthorship' + ' TEXT,\n'
query += 'canonicalName' + ' TEXT,\n'
query += 'genericName' + ' TEXT,\n'
query += 'specificEpithet' + ' TEXT,\n'
query += 'infraspecificEpithet' + ' TEXT,\n'
query += 'taxonRank' + " TEXT CHECK (taxonRank IN ('kingdom', 'phylum', 'class', 'order', 'family', 'genus', 'species', 'subspecies', 'variety')),\n"
query += 'nameAccordingTo' + ' TEXT,\n'
query += 'namePublishedIn' + ' TEXT,\n'
# Check what all of these taxonomic statuses can be--this check is failing
query += 'taxonomicStatus' + " TEXT CHECK (taxonomicStatus IN ('accepted', 'synonym', 'misapplied', 'doubtful')),\n"
query += 'nomenclaturalStatus' + ' TEXT,\n' 
query += 'taxonRemarks' + ' TEXT,\n'
query += 'kingdom' + ' TEXT,\n'
query += 'phylum' + ' TEXT,\n'
query += 'class' + ' TEXT,\n'
query += '"order"' + ' TEXT,\n'
query += 'family' + ' TEXT,\n'
query += 'genus' + ' TEXT);'

cur.execute(query)
conn.commit()
cur.close

<function cursor.close()>

<h1>IMPORTING NEW COLUMNS FROM CSV</h1>

In [5]:
# create psycopg2 connection
conn = psycopg2.connect(dbname='tpwd-tests',
                       user='postgres', host='localhost', port='5432',
                       password='postgres')
cur = conn.cursor()

# open csv, dump columns to matching names
with open('./Inverts_Taxon.csv', 'r') as f:
        next(f)
        cur.copy_expert(
                'COPY gbif_inverts_backbone FROM STDIN WITH CSV', f
        )
        
conn.commit()
cur.close

<function cursor.close()>

It's important to index these new tables. This improves substring searches WILDLY

In [ ]:
CREATE EXTENSION IF NOT EXISTS pg_trgm;

CREATE INDEX idx_canonical_name_trgm ON gbif_inverts_backbone USING GIN (canonicalname gin_trgm_ops)

In [ ]:
import zipfile



In [ ]:
with zipfile.ZipFile("C:\\Users\\mdw2685\\Desktop\\Projects\\early_tests\\GBIF_downloads\\0000133-250227182430271.zip", mode="r") as zf:
    files = zf.namelist()
    if "occurrence.txt" in files:
       zf.extract("occurrence.txt")